In [56]:
require 'nn';
require 'nngraph';
require 'math';

# Loading Data Module

In [57]:
DataLoader = {}
function DataLoader.split(input_file, sep,debug)
    --[[
        This method seperate the training samples from the label they have been given
    ]]
    sep = sep or '|'
    debug = debug or 0
    local x = {}
    local y = {}
    local w 
    f = assert(io.open(input_file,"r"))
    repeat 
    rawdata= f:read()
    if rawdata then 
        w = rawdata:split("|") 
        x[#x+1] = w[1]
        y[#y+1] = w[2]
    end
    until not rawdata
    return x,y
end

In [58]:

out_vocab_file = "/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7"
out_tensor_file = "/Users/david/Documents/MemoryNetwork/output_lua/data.t7"

In [59]:
function DataLoader.create_vocabulary(sample_tab,label_tab)
    local rawdata
    local max_sent_len = 0
    local sent_count = 0
    local unordered = {}
    for i=1,#sample_tab do 
        -- Writing on Sample File
        rawdata = sample_tab[i]
		for k,word in pairs(rawdata:split(" ")) do 
			word=word:lower()
			if not unordered[word] then unordered[word] = true end
		end
		sent_len = #rawdata:split(" ")
		if sent_len > max_sent_len then max_sent_len=sent_len end
        -- Writing on Label File
        rawdata = label_tab[i]
		for k,word in pairs(rawdata:split(" ")) do 
			word=word:lower()
			if not unordered[word] then unordered[word] = true end
		end
		sent_len = #rawdata:split(" ")
		if sent_len > max_sent_len then max_sent_len=sent_len end
		sent_count = sent_count + 1        
    end
	-- sort into a table (i.e. keys become 1..N)
	local ordered = {}
	for word in pairs(unordered) do ordered[#ordered + 1] = word end
	table.sort(ordered)
	-- invert `ordered` to create the char->int mapping
	local vocab_mapping = {}
	for i, word in ipairs(ordered) do
		vocab_mapping[word] = i
	end
	print('saving ' .. out_vocab_file)
    torch.save(out_vocab_file, vocab_mapping)
    return {sent_count,vocab_mapping,max_sent_len}
end

In [60]:
function DataLoader.create_tensor(sent_count,vocab_mapping,max_sent_len,input_tab,tensor_file)
	print('putting data into tensor...')
	local data = torch.ByteTensor(sent_count,max_sent_len):zero()
	local currline = 1
    local rawdata
    for i=1,#input_tab do 
        rawdata = input_tab[i]:lower()
		for k,word in pairs(rawdata:split(" ")) do 
			data[{currline,k}] = vocab_mapping[word:lower()]
		end
		currline = currline + 1
    end
	-- save output preprocessed files
    print(tensor_file)
    torch.save(tensor_file, data)
end


function DataLoader.text_to_tensor(data_dir, out_vocab_file, out_tensor_file)
    local timer = torch.Timer()
    local a,b = DataLoader.split(data_dir)
    local aa = DataLoader.create_vocabulary(a,b)
    
    local sent_count = aa[1]
    local vocab_mapping = aa[2]
    local max_sent_len = aa[3]
    local aaa = DataLoader.create_tensor(sent_count,vocab_mapping,max_sent_len,a,out_sample_tensor_file)
    local bbb = DataLoader.create_tensor(sent_count,vocab_mapping,max_sent_len,b,out_label_tensor_file)
    return aaa,bbb
end

# Testing the Loading Module

In [11]:
-- aa,bb = DataLoader.text_to_tensor(input_file,out_vocab_file,out_tensor_file)

In [12]:
sample_tensor = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/sample.t7")
label_tensor = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/label.t7")

In [13]:
sample_tensor:size()
label_tensor:size()

## Testing implemented component

In [9]:
dl = require 'utils.data_loader'

In [10]:
dl.text_to_tensor(input_file,out_vocab_file,out_tensor_file)

saving /Users/david/Documents/MemoryNetwork/output_lua/vocab.t7	


putting sample tensor into /Users/david/Documents/MemoryNetwork/output_lua/sample.t7...	


saving /Users/david/Documents/MemoryNetwork/output_lua/sample.t7	


putting sample tensor into /Users/david/Documents/MemoryNetwork/output_lua/label.t7...	
saving /Users/david/Documents/MemoryNetwork/output_lua/label.t7	


In [11]:
x = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/sample.t7")
y = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/label.t7")

## I Component

In [12]:
require 'nn'
require 'models.i_component'

In [13]:
x = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/sample.t7")
y = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/label.t7")
voc = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7")

In [20]:
-- We need a method to count the number of items in vocs because the implemented version stops at the first nil value
function count_table_elements(t)
    local count = 0
    for _ in pairs(t) do 
        count = count+1
    end 
    return count
end

In [21]:
voc_size = count_table_elements(voc)

In [16]:
i = IComponent(voc_size)

In [17]:
a = i:forward(x[1])

In [45]:
y = torch.Tensor(5):apply(function(x)
    i = 0
    i = i + 1
    return i
end)

In [ ]:
y = torch.Tensor(x:)

## Working with memory

## Playing around with data 

In [9]:
require 'nn';
require 'nngraph';

In [10]:
e = torch.Tensor(20):normal(0)

In [11]:
input = {}
table.insert(input,nn.Identity()())
lin = nn.Linear(20,100)(input[1])
r = nn.Reshape(4,25)(lin)
s = nn.SplitTable(1)(r)
output = {}
table.insert(output,s)-- placeholder)
mod = nn.gModule(input,output)

In [12]:
a = mod:forward(e)

In [13]:
bm = require 'models.base_model'

In [3]:
dl = require 'utils.data_loader'

In [22]:
-- dl.text_to_tensor(input_file,out_vocab_file,out_tensor_file)